In [1]:
import numpy as np
import pandas as pd

In [2]:
res1 = pd.read_csv('./submit/lgb_ccy.csv', encoding='utf-8')
res2 = pd.read_csv('./submit/sub_zy.csv', encoding='utf-8')
res3 = pd.read_csv('./submit/sub_lj.csv', encoding='utf-8')

lstm1 = pd.read_csv('./submit/fusai_lstm_1.csv', encoding='utf-8')
lstm2 = pd.read_csv('./submit/fusai_lstm_2.csv', encoding='utf-8')
res4 = lstm1[['id']]
res4['forecastVolum'] = list(map(lambda x, y: int((x + y) / 2), list(lstm1['forecastVolum']), list(lstm2['forecastVolum'])))

res1.columns = ['id', 'p1']
res2.columns = ['id', 'p2']
res3.columns = ['id', 'p3']
res4.columns = ['id', 'p4']

res = pd.merge(res1, res2, on='id', how='left')
res = res.merge(res3, on='id', how='left')
res = res.merge(res4, on='id', how='left')

pre_res = pd.read_csv('../chusai/submit/fusion_chusai.csv', encoding='utf-8').rename(columns={'forecastVolum': 'forecastVolum_2'})

sub_fusai = pd.read_csv('./data/evaluation_public.csv', encoding='utf-8').drop(['forecastVolum'], axis=1)
fusai = sub_fusai.merge(res, on='id', how='left').fillna(0)

print('ccy lgb:')
print('mean:', fusai['p1'].mean())
print(fusai.groupby(['regMonth'])['p1'].mean())
print('zy lgb:')
print('mean:', fusai['p2'].mean())
print(fusai.groupby(['regMonth'])['p2'].mean())
print('lj lgb:')
print('mean:', fusai['p3'].mean())
print(fusai.groupby(['regMonth'])['p3'].mean())
print('lstm:')
print('mean:', fusai['p4'].mean())
print(fusai.groupby(['regMonth'])['p4'].mean())

w11, w21, w31 = 0.4, 0.3, 0.3
w12, w22, w32 = 0.4, 0.3, 0.3
w13, w23, w33, w43 = 0.35, 0.2, 0.275, 0.1
w14, w24, w34, w44 = 0.35, 0.2, 0.275, 0.1

def model_fusion_function(value_1, value_2, value_3, value_4, month):
    if month == 1:
        return value_1 * w11 + value_2 * w21 + value_3 * w31
    elif month == 2:
        return value_1 * w12 + value_2 * w22 + value_3 * w32
    elif month == 3:
        return value_1 * w13 + value_2 * w23 + value_3 * w33 + w43 * value_4
    elif month == 4:
        return value_1 * w14 + value_2 * w24 + value_3 * w34 + w44 * value_4

fusai['forecastVolum'] = fusai.apply(lambda index: model_fusion_function(index.p1, index.p2, index.p3, index.p4, index.regMonth), axis=1)
fusai['forecastVolum'] = fusai['forecastVolum'].map(lambda index: int(np.round(index)))


fusai = fusai.merge(pre_res, on='id', how='left')
fusai['forecastVolum'] =  fusai.apply(lambda index: int(index.forecastVolum) if np.isnan(index.forecastVolum_2) else int(index.forecastVolum_2), axis=1)

print(fusai.describe()[['p1', 'p2', 'p3', 'p4']])
print('mean:', fusai['forecastVolum'].mean())
print(fusai.groupby(['regMonth'], as_index=False)['forecastVolum'].mean())

fusai[['id', 'forecastVolum']].to_csv('./submit/fusion_submit_final.csv', encoding='utf-8', index=None)
print('store over.')

ccy lgb:
mean: 48.35532150776053
regMonth
1    63.585366
2    46.189579
3    38.744457
4    44.901885
Name: p1, dtype: float64
zy lgb:
mean: 47.93819290465632
regMonth
1    67.180710
2    43.103659
3    37.261641
4    44.206763
Name: p2, dtype: float64
lj lgb:
mean: 47.57289356984479
regMonth
1    62.261086
2    43.666851
3    42.414080
4    41.949557
Name: p3, dtype: float64
lstm:
mean: 57.79628603104213
regMonth
1    66.710089
2    58.435144
3    55.812084
4    50.227827
Name: p4, dtype: float64
                p1           p2           p3           p4
count  7216.000000  7216.000000  7216.000000  7216.000000
mean     48.355322    47.938193    47.572894    57.796286
std     135.834956   143.009994   143.947966   172.533509
min       0.000000     0.000000     0.000000     0.000000
25%       0.000000     0.000000     0.000000     0.000000
50%       0.000000     0.000000     0.000000     0.000000
75%      22.000000    19.000000    17.000000    16.000000
max    2517.000000  1878.000000  